In [29]:
import pandas as pd
import tensorflow as tf
import nltk
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv('Downloads/train.csv')

In [4]:
df = df.dropna()

In [5]:
X = df.drop('label',axis = 1)
y = df['label']

In [6]:
voc_size = 5000
messages = X.copy()
messages.reset_index(inplace=True)

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/akshay/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
ps = PorterStemmer()
all_words =[]
for i in range(0, len(messages)):
    review =re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    all_words.append(review)

In [20]:
onehot_repr = [one_hot(words,voc_size)for words in all_words]

In [30]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
emb_vec_features = 40
model = Sequential()
model.add(Embedding(voc_size,emb_vec_features,input_length = sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss='binary_crossentropy',optimizer ='adam',metrics=['accuracy'])

In [31]:
X_f = np.array(embedded_docs)
y_f = np.array(y)
X_train, X_test, y_train,y_test=train_test_split(X_f,y_f,test_size=0.33, random_state=42)

In [32]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 4s 20ms/step - loss: 0.3388 - accuracy: 0.8389 - val_loss: 0.1956 - val_accuracy: 0.9165
Epoch 2/10
192/192 [==============================] - 3s 18ms/step - loss: 0.1493 - accuracy: 0.9425 - val_loss: 0.1863 - val_accuracy: 0.9254
Epoch 3/10
192/192 [==============================] - 3s 18ms/step - loss: 0.1082 - accuracy: 0.9598 - val_loss: 0.2111 - val_accuracy: 0.9215
Epoch 4/10
192/192 [==============================] - 4s 18ms/step - loss: 0.0846 - accuracy: 0.9700 - val_loss: 0.2455 - val_accuracy: 0.9173
Epoch 5/10
192/192 [==============================] - 4s 18ms/step - loss: 0.0728 - accuracy: 0.9738 - val_loss: 0.2624 - val_accuracy: 0.9181
Epoch 6/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0587 - accuracy: 0.9806 - val_loss: 0.2653 - val_accuracy: 0.9168
Epoch 7/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0476 - accuracy: 0.9844 - val_loss: 0.3462 - val_accuracy: 0.9171

In [33]:
y_pred = model.predict_classes(X_test)
confusion_matrix(y_test,y_pred),accuracy_score(y_test,y_pred)

(array([[3158,  261],
        [ 307, 2309]]),
 0.9058823529411765)